In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from tensorflow.keras.activations import relu
from tensorflow.keras.activations import sigmoid
import copy

In [6]:
def load_process_csv(csv_path):
    data = pd.read_csv(csv_path)
    photo_path = data.iloc[:,0]
    label = data.iloc[:,1].values
    photos_flat = []
    for paths in photo_path:
        img = Image.open(paths)
        img_array = np.array(img).reshape(-1)
        photos_flat.append(img_array)
    x=np.array(photos_flat).T/255
    y=label.reshape(1,-1)
    return x, y

In [7]:
train_x, train_y = load_process_csv(r"C:\Users\unarv\NURAL_NETWORK\train_dataset.csv")
validate_x, validate_y = load_process_csv(r"C:\Users\unarv\NURAL_NETWORK\validation_dataset.csv")
test_x, test_y = load_process_csv(r"C:\Users\unarv\NURAL_NETWORK\test_dataset.csv")

In [8]:
print(train_x.shape)

(49152, 601)


In [9]:
def initialize_parameter(dim_layer):
    np.random.seed(4)
    parameters={}
    L=len(dim_layer)
    for l in range(1,L):
        parameters['W'+str(l)] = np.random.randn(dim_layer[l],dim_layer[l-1])*0.01
        parameters['b'+str(l)] = np.zeros((dim_layer[l],1))
    return parameters

In [10]:
def linear_forward(A, W, b):
    Z=np.dot(W, A)+b
    cache = (A, W, b)
    return Z, cache

In [11]:
def linear_forward_activation(A, W, b, activation):
    if activation == 'sigmoid':
        Z, linear_cache = linear_forward(A, W, b)
        A = sigmoid(Z)
        activation_cache = Z
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A, W, b)
        A = relu(Z)
        activation_cache = Z
    cache = (linear_cache, activation_cache)
    return Z, cache

In [12]:
def forward(X,parameters):
    caches=[]
    A=X
    L = len(parameters)//2
    for l in range(1,L):
        A_prev=A
        A, cache = linear_forward_activation(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = 'relu')
        caches.append(cache)
    AL, cache = linear_forward_activation(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = 'sigmoid')
    caches.append(cache)
    return AL, caches

In [13]:
def comp_cost(AL, Y):
    m = Y.shape[1]
    epsilon = 1e-8
    AL = np.clip(AL, epsilon, 1 - epsilon)
    cost = -1/m*np.sum((Y*np.log(AL)+((1-Y)*np.log(1-AL))))
    cost = np.squeeze(cost)
    return cost

In [14]:
def linear_back(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = 1/m*np.dot(dZ,A_prev.T)
    db = 1/m*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    return dA_prev, dW, db

In [15]:
def sigmoid_backward(dA, activation_cache):
    Z = activation_cache
    s = sigmoid(Z)
    dZ = dA * s * (1 - s)
    return dZ
def relu_backward(dA, activation_cache):
    Z = activation_cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

In [16]:
def linear_backtivation(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_back(dZ, linear_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_back(dZ, linear_cache)
    return dA_prev, dW, db

In [17]:
def backward(AL, Y, caches):
    grads={}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_backtivation(dAL, current_cache, activation="sigmoid")
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_backtivation(grads["dA" + str(l + 1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads

In [18]:
def update_parameters(param, grads, learning_rate):
    parameters = copy.deepcopy(param)
    L = len(parameters) // 2
    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] -= learning_rate * grads["db" + str(l+1)]
    return parameters

In [19]:
dim_layer = [49152, 20, 7, 5, 1]

In [20]:
def deep_nn(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    np.random.seed(1)
    costs = []
    parameters = initialize_parameter(layers_dims)
    for i in range(0, num_iterations):
        AL, caches = forward(X, parameters)
        cost = comp_cost(AL, Y)
        grads = backward(AL, Y, caches)
        parameters  = update_parameters(parameters, grads, learning_rate)
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    return parameters, costs

In [ ]:
parameters, costs = deep_nn(train_x, train_y, dim_layer, num_iterations = 2500, print_cost = True)

Cost after iteration 0: 7.49843864781749
Cost after iteration 100: 9.409565706806616
Cost after iteration 200: 9.409565706806616
Cost after iteration 300: 9.409565706806616
Cost after iteration 400: 9.409565706806616
Cost after iteration 500: 9.409565706806616
Cost after iteration 600: 9.409565706806616
Cost after iteration 700: 9.409565706806616
Cost after iteration 800: 9.409565706806616
Cost after iteration 900: 9.409565706806616
Cost after iteration 1000: 9.409565706806616
Cost after iteration 1100: 9.409565706806616
Cost after iteration 1200: 9.409565706806616
Cost after iteration 1300: 9.409565706806616
Cost after iteration 1400: 9.409565706806616
Cost after iteration 1500: 9.409565706806616
Cost after iteration 1600: 9.409565706806616
Cost after iteration 1700: 9.409565706806616
Cost after iteration 1800: 9.409565706806616


In [ ]:
pred_train = predict(train_x, train_y, parameters)

In [ ]:
pred_test = predict(test_x, test_y, parameters)